# 颜色分拣玩法

### 导入头文件

In [1]:
#!/usr/bin/env python
# coding: utf-8
import cv2 as cv
import threading
from time import sleep
from dofbot_config import *
import ipywidgets as widgets
from IPython.display import display
from speech_identify_target import identify_GetTarget

### 创建实例,初始化参数

In [2]:
## 创建获取目标实例
target      = identify_GetTarget()
# 创建相机标定实例
calibration = Arm_Calibration()
# 初始化一些参数
num=0
dp    = []
xy=[90,135]
msg   = {}
threshold = 140
model = "General"
color_list = {}
# 初始化HSV值
color_hsv  = {"red"   : ((0, 43, 46), (10, 255, 255)),
              "green" : ((35, 43, 46), (77, 255, 255)),
              "blue"  : ((100, 43, 46), (124, 255, 255)),
              "yellow": ((21, 43, 46), (28, 255, 255))}
HSV_path="/home/yahboom/dofbot_ws/src/dofbot_color_identify/scripts/HSV_config.txt"
# XYT参数路径
XYT_path="/home/yahboom/dofbot_ws/src/dofbot_color_identify/scripts/XYT_config.txt"
try: read_HSV(HSV_path,color_hsv)
except Exception: print("Read HSV_config Error !!!")
try: xy, thresh = read_XYT(XYT_path)
except Exception: print("Read XYT_config Error !!!")

### 初始化机械臂位置

In [3]:
import Arm_Lib
# 创建机械臂驱动实例
import time
arm = Arm_Lib.Arm_Device()
joints_0 = [xy[0], xy[1], 0, 0, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建控件

In [4]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 调整滑杆
joint1_slider      = widgets.IntSlider(description='joint1 :'   ,    value=xy[0]     , min=70 , max=110, step=1, orientation='horizontal')
joint2_slider      = widgets.IntSlider(description='joint2 :'   ,    value=xy[1]     , min=115, max=155, step=1, orientation='horizontal')
threshold_slider   = widgets.IntSlider(description='threshold :',    value=threshold , min=0  , max=255, step=1, orientation='horizontal')

# 进入标定模式
calibration_model  = widgets.Button(description='calibration_model',  button_style='primary', layout=button_layout)
calibration_ok     = widgets.Button(description='calibration_ok',     button_style='success', layout=button_layout)
calibration_cancel = widgets.Button(description='calibration_cancel', button_style='danger', layout=button_layout)
# 选择抓取颜色
color_list_one     = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
color_list_two     = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
color_list_three   = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
color_list_four    = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
# 目标检测抓取
target_detection   = widgets.Button(description='target_detection', button_style='info', layout=button_layout)
reset_color_list   = widgets.Button(description='reset_color_list', button_style='info', layout=button_layout)
grap = widgets.Button(description='grap', button_style='success', layout=button_layout)
# 退出
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
color_down = widgets.HBox([exit_button], layout=widgets.Layout(align_self='center'));
color_img = widgets.VBox([imgbox, color_down], layout=widgets.Layout(align_self='center'));
color_identify = widgets.VBox(
    [joint1_slider, joint2_slider, threshold_slider, calibration_model, calibration_ok, calibration_cancel,
     color_list_one, color_list_two, color_list_three, color_list_four, target_detection, grap],
    layout=widgets.Layout(align_self='center'));
controls_box = widgets.HBox([color_img], layout=widgets.Layout(align_self='center'))

### 标定回调

In [5]:
def calibration_model_Callback(value):
    global model
    model = 'Calibration'
    with output: print(model)
def calibration_OK_Callback(value):
    global model
    model = 'calibration_OK'
    with output: print(model)
def calibration_cancel_Callback(value):
    global model
    model = 'calibration_Cancel'
    with output: print(model)
calibration_model.on_click(calibration_model_Callback)
calibration_ok.on_click(calibration_OK_Callback)
calibration_cancel.on_click(calibration_cancel_Callback)

### 颜色选择序列

In [6]:
# 选择颜色
def color_list_one_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '1' in color_list:del color_list['1']
    elif value['new'] == "red":
        color_list['1'] = "red"
    elif value['new']== "green":
        color_list['1'] = "green"
    elif value['new'] == "blue":
        color_list['1'] = "blue"
    elif value['new'] == "yellow":
        color_list['1'] = "yellow"
    with output:
        print("color_list_three_Callback clicked.",color_list)
def color_list_two_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '2' in color_list:del color_list['2']
    elif value['new'] == "red":
        color_list['2'] = "red"
    elif value['new'] == "green":
        color_list['2'] = "green"
    elif value['new'] == "blue":
        color_list['2'] = "blue"
    elif value['new'] == "yellow":
        color_list['2'] = "yellow"
    with output:
        print("color_list_three_Callback clicked.",color_list)
def color_list_three_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '3' in color_list:del color_list['3']
    elif value['new'] == "red":
        color_list['3'] = "red"
    elif value['new'] == "green":
        color_list['3'] = "green"
    elif value['new'] == "blue":
        color_list['3'] = "blue"
    elif value['new'] == "yellow":
        color_list['3'] = "yellow"
    with output:
        print("color_list_three_Callback clicked.",color_list)
def color_list_four_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '4' in color_list:del color_list['4']
    elif value['new'] == "red":
        color_list['4'] = "red"
    elif value['new'] == "green":
        color_list['4'] = "green"
    elif value['new'] == "blue":
        color_list['4'] = "blue"
    elif value['new'] == "yellow":
        color_list['4'] = "yellow"
    with output:
        print("color_list_four_Callback clicked.",color_list)
color_list_one.observe(color_list_one_Callback)
color_list_two.observe(color_list_two_Callback)
color_list_three.observe(color_list_three_Callback)
color_list_four.observe(color_list_four_Callback)

### 抓取控制

In [7]:
# 抓取控制
def target_detection_Callback(value):
    global model
    model = 'Detection'
    with output: print(model)
def reset_color_list_Callback(value):
    global model
    model = 'Reset_list'
    with output: print(model)
def grap_Callback(value):
    global model
    model = 'Grap'
    with output: print(model)
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
target_detection.on_click(target_detection_Callback)
reset_color_list.on_click(reset_color_list_Callback)
grap.on_click(grap_Callback)
exit_button.on_click(exit_button_Callback)

### 主程序

In [8]:
def camera():
    global color_hsv,model,dp,msg,color_list,color_read
    # 打开摄像头
    capture = cv.VideoCapture(0)
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            result = arm.Arm_serial_speech_read(1)
            if (result != 0):
                print(result)
            # 读取相机的每一帧
            _, img = capture.read()
            # 统一图像大小
            img = cv.resize(img, (640, 480))
            xy=[joint1_slider.value,joint2_slider.value]
            if model == 'Calibration':
                time.sleep(0.5)
                _, img = calibration.calibration_map(img,xy,threshold_slider.value)
                model = 'calibration_OK'
            if model == 'calibration_OK':
                time.sleep(0.5)
                try: write_XYT(XYT_path,xy, threshold_slider.value)
                except Exception: print("File XYT_config Error !!!")
                dp, img = calibration.calibration_map(img,xy,threshold_slider.value)
                model="Detection2"
            if len(dp) != 0: img = calibration.Perspective_transform(dp, img)
            if model == 'calibration_Cancel':  
                dp = []
                msg= {}
                color_read = 4
                model="General"
            if model == 'Detection2':
                _, img = capture.read()
                # 统一图像大小
                img = cv.resize(img, (640, 480))
                xy=[joint1_slider.value,joint2_slider.value]
                color_read = 4
                color_list['1'] = "red"
                color_list['2'] = "green"
                color_list['3'] = "blue"
                color_list['4'] = "yellow"
                img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                time.sleep(0.1)
                # Speech_text("识别到",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.1)  
                # if coo4 == 'yellow':
                #     color_read += 1
                #     arm.Arm_ask_speech(31)
                #     # Speech_text("黄色",EncodingFormat_Type["GB2312"])
                #     # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.6)  
                # if coo1 == 'red':
                #     color_read += 1
                #     arm.Arm_ask_speech(33)
                #     # Speech_text("红色",EncodingFormat_Type["GB2312"])
                #     # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.6)   
                # if coo2 == 'green':
                #     color_read += 1
                #     arm.Arm_ask_speech(34)
                #     # Speech_text("绿色",EncodingFormat_Type["GB2312"])
                #     # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.6)  
                # if coo3 == 'blue':
                #     color_read += 1
                #     arm.Arm_ask_speech(35)
                #     # Speech_text("蓝色",EncodingFormat_Type["GB2312"])
                #     # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.6)  
                
                imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
                arm.Arm_ask_speech(36)
                # Speech_text("第一个要排序的颜色是：",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                time.sleep(.1)  
                model="list1"
               
            if model == 'list1' and color_read > 0:  
                if result == 43:
                    # print(result)
                    color_list['1'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist1"
                elif result == 44:
                    # print(result)
                    color_list['1'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist1"
                elif result == 45:
                    # print(result)
                    color_list['1'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist1"
                elif result == 33:
                    # print(result)
                    color_list['1'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist1"
                elif result == 35:
                    # print(result)
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist1"
                    
            if model == 'tolist1':
                # print(result)
                if color_read > 0:
                    arm.Arm_ask_speech(37)
                    # Speech_text("好的，第二个要排序的颜色是：",EncodingFormat_Type["GB2312"])
                    # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                    time.sleep(.1) 
                    result = 0
                    model="list2"
                elif color_read == 0:
                    arm.Arm_ask_speech(38)
                    # Speech_text("好的，排序完成，是否开始分拣",EncodingFormat_Type["GB2312"])
                    # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                    time.sleep(.1) 
                    model="toGrap"
                     
            if model == 'list2' and color_read > 0:   
                
                if result == 43:
                    # print(result)
                    color_list['2'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist2"
                elif result == 44:
                    # print(result)
                    color_list['2'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist2"
                elif result == 45:
                    # print(result)
                    color_list['2'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist2"
                elif result == 33:
                    # print(result)
                    color_list['2'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist2"
                elif result == 35:
                    # print(result)
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist2"
                    
            if model == 'tolist2':
                # print(result)
                if color_read > 0:
                    arm.Arm_ask_speech(39)
                    # Speech_text("好的，第三个要排序的颜色是：",EncodingFormat_Type["GB2312"])
                    # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                    time.sleep(.1) 
                    model="list3"
                elif color_read == 0:
                    arm.Arm_ask_speech(38)
                    # Speech_text("好的，排序完成，是否开始分拣",EncodingFormat_Type["GB2312"])
                    # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                    time.sleep(.1) 
                    model="toGrap"
           
            if model == 'list3' and color_read > 0:  
                if result == 43:
                    # print(result)
                    color_list['3'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist3"
                elif result == 44:
                    # print(result)
                    color_list['3'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist3"
                elif result == 45:
                    # print(result)
                    color_list['3'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist3"
                elif result == 33:
                    # print(result)
                    color_list['3'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist3"
                elif result == 35:
                    # print(result)
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist3"
            if model == 'tolist3':
                if color_read > 0:
                    # print(result)
                    arm.Arm_ask_speech(40)
                    # Speech_text("好的，第四个要排序的颜色是：",EncodingFormat_Type["GB2312"])
                    # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                    time.sleep(.1) 
                    model="list4"
                elif color_read == 0:
                    arm.Arm_ask_speech(38)
                    # Speech_text("好的，排序完成，是否开始分拣",EncodingFormat_Type["GB2312"])
                    # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                    time.sleep(.1) 
                    model="toGrap"
                print(color_read)              
            if model == 'list4' and color_read > 0:  
                if result == 43:
                    # print(result)
                    color_list['4'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist4"
                elif result == 44:
                    # print(result)
                    color_list['4'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist4"
                elif result == 45:
                    color_list['4'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist4"
                elif result == 33:
                    # print(result)
                    color_list['4'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist4"
                elif result == 35:
                    # print(result)
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                    result = 0
                    color_read -= 1
                    model="tolist4"
                  
            if model == 'tolist4':
                # print(result)
                arm.Arm_ask_speech(38)
                # Speech_text("好的，排序完成，是否开始分拣",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                time.sleep(.1) 
                model="toGrap"
                  
            if model == 'toGrap':  
                img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                #coo = {}
                #img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv,color_list)
                if result == 34:
                    result = 0
                    model="Grap"
            
            if result == 36:
                result = 0
                msg={}
                color_list = {}
                model="Detection2"
                # Speech_text("好的",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                time.sleep(0.1) 
                
            if model=="Reset_list":
                # print(coo1)
                # print(coo2)
                # print(coo3)
                # print(coo4)
                msg={}
                color_list = {}
                model="General"
            if len(msg)!= 0 and model == 'Grap':
                threading.Thread(target=target.target_run, args=(msg,xy)).start()
                msg={}
                model="General"
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动

In [9]:
# display(controls_box)
#第一步先自动标定
model = 'Calibration'
display(controls_box,output)
threading.Thread(target=camera, ).start()

Output()

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (935) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


43
44
35
1
35
35
